In [ ]:
from __future__ import print_function
import os
import time
import logging
import random
import argparse
import numpy as np
from visdom import Visdom
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *
from metric.loss import FitNet, AttentionTransfer, RKdAngle, RkdDistance

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
import models

# Student models:
# myNet, LeNet, FitNet

start_time = time.time()

# Training settings
parser = argparse.ArgumentParser(description='PyTorch LR_adaptive_AT')

parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100'
                            ],
                    default='CIFAR10')
parser.add_argument('--teachers',
                    choices=['ResNet32',
                             'ResNet44',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default=['ResNet44', 'ResNet56', 'ResNet110'],
                    nargs='+')
parser.add_argument('--student',
                    choices=['ResNet20',
                             'myNet'
                            ],
                    default='ResNet20')
parser.add_argument('--kd_ratio', default=0.7, type=float)
parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--T', type=float, default=20.0, metavar='Temputure', help='Temputure for distillation')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:1', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=10, metavar='N', help='how many batches to wait before logging training status')

config = ['--epochs', '200', '--teachers', 'ResNet32', 'ResNet56', 'ResNet110', '--T', '5.0', '--device', 'cuda:1']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
load_dir = './checkpoint/' + args.dataset + '/'

# teachers model
teacher_models = []
for te in args.teachers:
    te_model = getattr(models, te)(num_classes=args.n_class)
#     print(te_model)
    te_model.load_state_dict(torch.load(load_dir + te_model.model_name + '.pth'))
    te_model.to(device)
    te_model.eval()  # eval mode
    teacher_models.append(te_model)

st_model = getattr(models, args.student)()  # args.student()
st_model.to(device)

# logging
logfile = load_dir + 'adapter2_distill_' + st_model.model_name + '.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.array([0]),
    Y=np.array([0]),
    opts=dict(
        title='adapter + multi-AT Loss',
        xlabel='epoch',
        xtickmin=0,
#         xtickmax=1,
#         xtickstep=5,
        ylabel='loss',
        ytickmin=0,
#         ytickmax=1,
        ytickstep=0.5,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
    ),
    name="loss"
)

acc_win = vis.line(
    X=np.column_stack((0, 0)),
    Y=np.column_stack((0, 0)),
    opts=dict(
        title='adapter + multi-AT Acc',
        xlabel='epoch',
        xtickmin=0,
#         xtickstep=5,
        ylabel='accuracy',
        ytickmin=0,
        ytickmax=100,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
        legend=['train_acc', 'test_acc']
    ),
    name="acc"
)

# get triplets
def random_triplets(st_maps, te_maps):
    # input: t1, t2 - triplet pair
    def triplet_distance(t1, t2):
        return (t1 - t2).pow(2).sum()
    
    conflict = 0
    st_triplet_list = []
    triplet_set_size = st_maps.size(0)
    batch_list = [x for x in range(triplet_set_size)]
    for i in range(triplet_set_size):
        triplet_index = random.sample(batch_list, 3)
        anchor_index = triplet_index[0]  # denote the 1st triplet item as anchor
        st_triplet = st_maps[triplet_index]
        te_triplet = te_maps[triplet_index]
        distance_01 = triplet_distance(te_triplet[0], te_triplet[1])
        distance_02 = triplet_distance(te_triplet[0], te_triplet[2])
        if distance_01 > distance_02:
            conflict += 1
            # swap postive and negative
            st_triplet[1], st_triplet[2] = st_triplet[2], st_triplet[1]
        st_triplet_list.append(st_triplet)
    
    st_triplet_batch = torch.stack(st_triplet_list, dim=1)
    return st_triplet_batch


# adapter model
class Adapter():
    def __init__(self, in_models, pool_size):
        # representations of teachers
        pool_ch = pool_size[1]  # 64
        pool_w = pool_size[2]   # 8
        LR_list = []
        torch.manual_seed(1)
        self.theta = torch.randn(len(in_models), pool_ch).to(device)  # [3, 64]
        self.theta.requires_grad_(True)
   
        self.max_feat = nn.MaxPool2d(kernel_size=(pool_w, pool_w), stride=pool_w).to(device)
        self.W = torch.randn(pool_ch, 1).to(device)
        self.W.requires_grad_(True)
        self.val = False

    def loss(self, y, labels, weighted_logits, T=10.0, alpha=0.7):
        ls = nn.KLDivLoss()(F.log_softmax(y/T), weighted_logits) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)
        if not self.val:
            ls += 0.1 * (torch.sum(self.W * self.W) + torch.sum(torch.sum(self.theta * self.theta, dim=1), dim=0))
        return ls
        
    def gradient(self, lr=0.01):
        self.W.data = self.W.data - lr * self.W.grad.data
        # Manually zero the gradients after updating weights
        self.W.grad.data.zero_()
        
    def eval(self):
        self.val = True
        self.theta.detach()
        self.W.detach()
    
    # input size: [64, 8, 8], [128, 3, 10]
    def forward(self, conv_map, te_logits_list):
        beta = self.max_feat(conv_map)
        beta = torch.squeeze(beta)  # [128, 64]
        
        latent_factor = []
        for t in self.theta:
            latent_factor.append(beta * t)
#         latent_factor = torch.stack(latent_factor, dim=0)  # [3, 128, 64]
        alpha = []
        for lf in latent_factor:  # lf.size:[128, 64]
            alpha.append(lf.mm(self.W))
        alpha = torch.stack(alpha, dim=0)  # [3, 128, 1]
        alpha = torch.squeeze(alpha).transpose(0, 1) # [128, 3]
        miu = F.softmax(alpha)  # [128, 3]
        miu = torch.unsqueeze(miu, dim=2)
        weighted_logits = miu * te_logits_list  # [128, 3, 10]
        weighted_logits = torch.sum(weighted_logits, dim=1)
#         print(weighted_logits)
        
        return weighted_logits

# adapter instance
_,_,_,pool_m,_ = st_model(torch.randn(1,3, 128, 128).to(device))  # get pool_size of student
# reate adapter instance
adapter = Adapter(teacher_models, pool_m.size())


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
train_set = getattr(datasets, args.dataset)(root='../data', train=True, download=True, transform=train_transform)
test_set = getattr(datasets, args.dataset)(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)
# optim
optimizer_W = optim.SGD([adapter.W], lr=args.lr, momentum=0.9)
optimizer_theta = optim.SGD([adapter.theta], lr=args.lr, momentum=0.9)
optimizer_sgd = optim.SGD(st_model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_sgd, gamma=0.1, milestones=[100, 150])
lr_scheduler2 = optim.lr_scheduler.MultiStepLR(optimizer_W, milestones=[40, 50])
lr_scheduler3 = optim.lr_scheduler.MultiStepLR(optimizer_theta, milestones=[40, 50])

# losses
dist_criterion = RkdDistance().to(device)
angle_criterion = RKdAngle().to(device)
# triplet loss
triplet_loss = nn.TripletMarginLoss(margin=0.2, p=2).to(device)


def train_adapter(n_epochs=70, model=st_model):
    print('Training adapter:')
    start_time = time.time()
    model.train()

    for ep in range(n_epochs):
        lr_scheduler2.step()
        lr_scheduler3.step()
        for i, (input, target) in enumerate(train_loader):

            input, target = input.to(device), target.to(device)
            # compute outputs
            b1, b2, b3, pool, output = model(input) # out_feat: 16, 32, 64, 64, - 
            st_maps = [b1, b2, b3, pool]
#             print('b1:{}, b2:{}, b3{}, pool:{}'.format(b1.size(), b2.size(), b3.size(), pool.size()))

            te_scores_list = []
            hint_maps = []
            for j,te in enumerate(teacher_models):
#                 te.eval()
                with torch.no_grad():
                    t_b1, t_b2, t_b3, t_pool, t_output = te(input)
                
#                 print('t_b1:{}, t_b2:{}, t_b3{}, t_pool:{}'.format(t_b1.size(), t_b2.size(), t_b3.size(), t_pool.size()))
                hint_maps.append([t_b1, t_b2, t_b3, t_pool])
                t_output = F.softmax(t_output/args.T)
                te_scores_list.append(t_output)
            te_scores_Tensor = torch.stack(te_scores_list, dim=1)  # size: [128, 3, 10]
            
            optimizer_sgd.zero_grad()
            optimizer_W.zero_grad()
            optimizer_theta.zero_grad()
            
            st_tripets = random_triplets(b2, t_b2)
            relation_loss = triplet_loss(st_tripets[0], st_tripets[1], st_tripets[2])
            
            weighted_logits = adapter.forward(pool, te_scores_Tensor)
            
            angle_loss = angle_criterion(output, weighted_logits)
            dist_loss = dist_criterion(output, weighted_logits)
            # compute gradient and do SGD step
            ada_loss = adapter.loss(output, target, weighted_logits, T=args.T, alpha=args.kd_ratio)
            loss = ada_loss + angle_loss + dist_loss + relation_loss
            
            loss.backward(retain_graph=True)
            optimizer_sgd.step()
            optimizer_W.step()
            optimizer_theta.step()
            
#          vis.line(np.array([loss.item()]), np.array([ep]), loss_win, update="append")
        log_out('epoch[{}/{}]adapter Loss: {:.4f}'.format(ep, n_epochs, loss.item()))
    end_time = time.time()
    log_out("--- adapter training cost {:.3f} mins ---".format((end_time - start_time)/60))


# train with multi-teacher
def train(epoch, model):
    print('Training:')
    # switch to train mode
    model.train()
    adapter.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        
        # compute outputs
        b1, b2, b3, pool, output = model(input)
        st_maps = [b1, b2, b3, pool]
        
        te_scores_list = []
        hint_maps = []
        for j,te in enumerate(teacher_models):
            te.eval()
            t_b1, t_b2, t_b3, t_pool, t_output = te(input)
            t_b1, t_b2, t_b3, t_pool, t_output = t_b1.detach(), t_b2.detach(), t_b3.detach(), t_pool.detach(), t_output.detach()
            hint_maps.append([t_b1, t_b2, t_b3, t_pool])
            t_output = F.softmax(t_output/args.T)
            te_scores_list.append(t_output)
        te_scores_Tensor = torch.stack(te_scores_list, dim=1)  # size: [128, 3, 10]
        weighted_logits = adapter.forward(pool, te_scores_Tensor)
        
        optimizer_sgd.zero_grad()
        
        angle_loss = angle_criterion(output, weighted_logits)
        dist_loss = dist_criterion(output, weighted_logits)
        
        st_tripets = random_triplets(b2, t_b2)
        relation_loss = triplet_loss(st_tripets[0], st_tripets[1], st_tripets[2])

        weighted_logits = adapter.forward(pool, te_scores_Tensor)
        # compute gradient and do SGD step
        ada_loss = adapter.loss(output, target, weighted_logits, T=args.T, alpha=args.kd_ratio)
        loss = ada_loss + angle_loss + dist_loss + relation_loss

        loss.backward(retain_graph=True)
        optimizer_sgd.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        train_acc = accuracy(output.data, target.data)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(train_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg, train_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_,output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()

# """
print('StudentNet:\n')
print(st_model)
st_model.apply(weights_init_normal)
train_adapter(n_epochs=80)
# st_model.apply(weights_init_normal)
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
    log_out('current lr {:.5e}'.format(optimizer_sgd.param_groups[0]['lr']))
    lr_scheduler.step(epoch)
    train_loss, train_acc = train(epoch, st_model)
    # visaulize loss
    vis.line(np.array([train_loss]), np.array([epoch]), loss_win, update="append")
    _, test_acc, top1 = test(st_model)
    vis.line(np.column_stack((train_acc, top1)), np.column_stack((epoch, epoch)), acc_win, update="append")
    if top1 > best_acc:
        best_acc = top1
        
# release GPU memory
torch.cuda.empty_cache()
log_out("BEST ACC: {:.3f}".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))
# """

/home/data/yaliu/jupyterbooks/multi-KD/models/teacher/resnet20.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
StudentNet:

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:277: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:210: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/_reduction.py:15: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:181: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/fu

epoch[0/80]adapter Loss: 2.6623
epoch[1/80]adapter Loss: 1.8386
epoch[2/80]adapter Loss: 1.7279
epoch[3/80]adapter Loss: 1.4636
epoch[4/80]adapter Loss: 1.2764
epoch[5/80]adapter Loss: 1.3325
epoch[6/80]adapter Loss: 1.2382
epoch[7/80]adapter Loss: 1.1367
epoch[8/80]adapter Loss: 1.3161
epoch[9/80]adapter Loss: 1.0568
epoch[10/80]adapter Loss: 1.0053
epoch[11/80]adapter Loss: 1.1947
epoch[12/80]adapter Loss: 0.6687
epoch[13/80]adapter Loss: 1.2803
epoch[14/80]adapter Loss: 1.0175
epoch[15/80]adapter Loss: 0.9144
epoch[16/80]adapter Loss: 0.7287
epoch[17/80]adapter Loss: 1.0714
epoch[18/80]adapter Loss: 1.0551
epoch[19/80]adapter Loss: 0.9994
epoch[20/80]adapter Loss: 0.9893
epoch[21/80]adapter Loss: 0.7603
epoch[22/80]adapter Loss: 0.8927
epoch[23/80]adapter Loss: 0.8880
epoch[24/80]adapter Loss: 1.0651
epoch[25/80]adapter Loss: 1.0390
epoch[26/80]adapter Loss: 0.6986
epoch[27/80]adapter Loss: 0.7813
epoch[28/80]adapter Loss: 0.7678
epoch[29/80]adapter Loss: 0.8396
epoch[30/80]adapter 

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:337: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

[340/391]	Time 0.128 (0.133)	Data 0.018 (0.019)	Loss 0.7386 (0.7583)	Prec@1 86.719 (85.754)
[350/391]	Time 0.130 (0.133)	Data 0.019 (0.019)	Loss 0.7558 (0.7588)	Prec@1 82.031 (85.759)
[360/391]	Time 0.131 (0.133)	Data 0.019 (0.019)	Loss 0.7502 (0.7588)	Prec@1 86.719 (85.786)
[370/391]	Time 0.128 (0.133)	Data 0.018 (0.019)	Loss 0.7388 (0.7592)	Prec@1 89.844 (85.782)
[380/391]	Time 0.130 (0.133)	Data 0.019 (0.019)	Loss 0.8976 (0.7599)	Prec@1 83.594 (85.788)
[390/391]	Time 0.086 (0.133)	Data 0.012 (0.019)	Loss 0.6997 (0.7617)	Prec@1 86.250 (85.724)
Testing:
Test: [0/79]	Time 0.019 (0.019)	Loss 1.2832 (1.2832)	Prec@1 71.094 (71.094)
Test: [10/79]	Time 0.020 (0.019)	Loss 1.0630 (1.2739)	Prec@1 75.781 (74.432)
Test: [20/79]	Time 0.019 (0.019)	Loss 1.2429 (1.3507)	Prec@1 75.000 (73.103)
Test: [30/79]	Time 0.020 (0.019)	Loss 1.5388 (1.3696)	Prec@1 71.094 (72.681)
Test: [40/79]	Time 0.019 (0.019)	Loss 1.3100 (1.3804)	Prec@1 72.656 (72.694)
Test: [50/79]	Time 0.020 (0.019)	Loss 1.3903 (1.3866)	P

[290/391]	Time 0.132 (0.133)	Data 0.019 (0.020)	Loss 0.8031 (0.7434)	Prec@1 85.156 (86.174)
[300/391]	Time 0.130 (0.133)	Data 0.019 (0.020)	Loss 0.6541 (0.7437)	Prec@1 89.844 (86.174)
[310/391]	Time 0.130 (0.133)	Data 0.018 (0.020)	Loss 0.8841 (0.7442)	Prec@1 85.156 (86.143)
[320/391]	Time 0.131 (0.133)	Data 0.018 (0.020)	Loss 0.7673 (0.7455)	Prec@1 85.938 (86.122)
[330/391]	Time 0.129 (0.133)	Data 0.018 (0.020)	Loss 0.8103 (0.7469)	Prec@1 85.156 (86.093)
[340/391]	Time 0.129 (0.133)	Data 0.018 (0.020)	Loss 0.7321 (0.7493)	Prec@1 84.375 (86.013)
[350/391]	Time 0.130 (0.133)	Data 0.018 (0.020)	Loss 0.7395 (0.7503)	Prec@1 86.719 (85.980)
[360/391]	Time 0.141 (0.133)	Data 0.028 (0.020)	Loss 0.6959 (0.7506)	Prec@1 88.281 (85.985)
[370/391]	Time 0.145 (0.133)	Data 0.024 (0.020)	Loss 0.7907 (0.7503)	Prec@1 85.938 (86.018)
[380/391]	Time 0.139 (0.133)	Data 0.019 (0.019)	Loss 0.8464 (0.7508)	Prec@1 78.906 (86.011)
[390/391]	Time 0.084 (0.133)	Data 0.012 (0.019)	Loss 0.8084 (0.7504)	Prec@1 88.7

[240/391]	Time 0.128 (0.132)	Data 0.018 (0.019)	Loss 0.7870 (0.7548)	Prec@1 88.281 (85.999)
[250/391]	Time 0.134 (0.132)	Data 0.019 (0.019)	Loss 0.7666 (0.7562)	Prec@1 86.719 (85.944)
[260/391]	Time 0.132 (0.132)	Data 0.019 (0.019)	Loss 0.6153 (0.7574)	Prec@1 92.188 (85.908)
[270/391]	Time 0.137 (0.132)	Data 0.020 (0.019)	Loss 0.7330 (0.7567)	Prec@1 86.719 (85.923)
[280/391]	Time 0.133 (0.132)	Data 0.019 (0.019)	Loss 0.6341 (0.7557)	Prec@1 91.406 (85.938)
[290/391]	Time 0.138 (0.132)	Data 0.022 (0.019)	Loss 0.9630 (0.7554)	Prec@1 80.469 (85.956)
[300/391]	Time 0.151 (0.133)	Data 0.025 (0.019)	Loss 0.8296 (0.7556)	Prec@1 82.031 (85.899)
[310/391]	Time 0.138 (0.133)	Data 0.020 (0.019)	Loss 0.8835 (0.7570)	Prec@1 80.469 (85.865)
[320/391]	Time 0.132 (0.133)	Data 0.019 (0.019)	Loss 0.8709 (0.7567)	Prec@1 79.688 (85.855)
[330/391]	Time 0.131 (0.133)	Data 0.019 (0.019)	Loss 0.8517 (0.7565)	Prec@1 81.250 (85.862)
[340/391]	Time 0.128 (0.133)	Data 0.019 (0.019)	Loss 0.7977 (0.7563)	Prec@1 85.9

[190/391]	Time 0.133 (0.133)	Data 0.019 (0.019)	Loss 0.8964 (0.7586)	Prec@1 81.250 (85.659)
[200/391]	Time 0.133 (0.133)	Data 0.019 (0.019)	Loss 0.7911 (0.7599)	Prec@1 81.250 (85.623)
[210/391]	Time 0.135 (0.133)	Data 0.020 (0.019)	Loss 0.7649 (0.7617)	Prec@1 85.938 (85.612)
[220/391]	Time 0.133 (0.133)	Data 0.019 (0.019)	Loss 0.8240 (0.7624)	Prec@1 82.031 (85.616)
[230/391]	Time 0.131 (0.133)	Data 0.019 (0.019)	Loss 0.7457 (0.7620)	Prec@1 85.938 (85.653)
[240/391]	Time 0.135 (0.133)	Data 0.020 (0.019)	Loss 0.6940 (0.7614)	Prec@1 87.500 (85.668)
[250/391]	Time 0.131 (0.133)	Data 0.019 (0.019)	Loss 0.7617 (0.7615)	Prec@1 85.938 (85.673)
[260/391]	Time 0.135 (0.133)	Data 0.020 (0.019)	Loss 0.6900 (0.7606)	Prec@1 86.719 (85.674)
[270/391]	Time 0.131 (0.133)	Data 0.019 (0.019)	Loss 0.6518 (0.7593)	Prec@1 87.500 (85.704)
[280/391]	Time 0.132 (0.133)	Data 0.019 (0.019)	Loss 0.7457 (0.7595)	Prec@1 89.062 (85.698)
[290/391]	Time 0.133 (0.133)	Data 0.020 (0.019)	Loss 0.6955 (0.7586)	Prec@1 85.1

[140/391]	Time 0.131 (0.133)	Data 0.020 (0.019)	Loss 0.7170 (0.7503)	Prec@1 88.281 (85.993)
[150/391]	Time 0.130 (0.133)	Data 0.018 (0.019)	Loss 0.7098 (0.7487)	Prec@1 88.281 (86.124)
[160/391]	Time 0.129 (0.133)	Data 0.018 (0.019)	Loss 0.7354 (0.7497)	Prec@1 89.062 (86.146)
[170/391]	Time 0.134 (0.132)	Data 0.018 (0.019)	Loss 0.6483 (0.7513)	Prec@1 88.281 (86.043)
[180/391]	Time 0.130 (0.132)	Data 0.020 (0.019)	Loss 0.6727 (0.7519)	Prec@1 88.281 (86.041)
[190/391]	Time 0.131 (0.132)	Data 0.018 (0.019)	Loss 0.7834 (0.7534)	Prec@1 85.938 (85.999)
[200/391]	Time 0.137 (0.132)	Data 0.023 (0.019)	Loss 0.6503 (0.7524)	Prec@1 90.625 (86.019)
[210/391]	Time 0.129 (0.132)	Data 0.018 (0.019)	Loss 0.8770 (0.7526)	Prec@1 84.375 (86.026)
[220/391]	Time 0.130 (0.132)	Data 0.018 (0.019)	Loss 0.5814 (0.7514)	Prec@1 92.188 (86.068)
[230/391]	Time 0.137 (0.132)	Data 0.019 (0.019)	Loss 0.7420 (0.7519)	Prec@1 87.500 (86.039)
[240/391]	Time 0.131 (0.132)	Data 0.020 (0.019)	Loss 0.6965 (0.7511)	Prec@1 91.4